In [4]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [5]:
import warnings
warnings.simplefilter('ignore') #ignores simple warning

In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
#Read data
datav10 = xr.open_mfdataset('v10\*.nc',combine = 'by_coords')
datau10 = xr.open_mfdataset('u10\*.nc',combine = 'by_coords')
datamsl = xr.open_mfdataset('mslp\*.nc',combine = 'by_coords')/100
datav925 = xr.open_mfdataset('v925\*.nc',combine = 'by_coords')
datau925 = xr.open_mfdataset('u925\*.nc',combine = 'by_coords')

In [8]:
#CENS Identification
#Averaged v10 (105-110 BT, 0-5 LS) < mean-std
censidx=datav10.sel(lat= slice(-5,0), lon=slice(105,110), time=slice('1979-11-01T00:00:00.000000000','2020-03-31T00:00:00.000000000'))
censidx=censidx.rolling(time=3, center=True).mean()
censidx=censidx.sel(time=censidx.time.dt.season=="DJF")
censidx=censidx.mean(dim=('lat', 'lon'))

#Time data
time=censidx.variables['time'].values

#Change to np array
#Calculate threshold
vcens=censidx.variables['v10'].values
th=np.mean(vcens)-1*np.std(vcens)
print('threshold CENS idx= ', th, 'm/s')

#Event CENS --> vcens<th
censevt=(vcens<th)*1

threshold CENS idx=  -4.158942103385925 m/s


In [9]:
#CS no-CS Classification
#u<=0
ucs=datau925.sel(lat= slice(5,10), lon=slice(107,115), time=slice('1979-12-01T00:00:00.000000000','2020-02-29T00:00:00.000000000'))
ucs=ucs.sel(time=ucs.time.dt.season=="DJF")['u'].values
ucsidx=np.mean(np.mean(ucs, axis=2), axis=1)<=0

#v<0
vcs=datav925.sel(lat= slice(5,10), lon=slice(107,115), time=slice('1979-12-01T00:00:00.000000000','2020-02-29T00:00:00.000000000'))
vcs=vcs.sel(time=vcs.time.dt.season=="DJF")['v'].values
vcsidx=np.mean(np.mean(vcs, axis=2), axis=1)<0

#windspeed>=mean+0.5 std
wd=np.sqrt(ucs**2+vcs**2)
wd=np.mean(np.mean(wd, axis=2), axis=1)
thwd=(np.mean(wd)+0.5*np.std(wd))
wdidx=wd>=thwd
print('threshold CS wind idx= ', thwd, 'm/s')
wdcsidx=(vcsidx*ucsidx*wdidx)

#MSLP criteria (18-22 LU, 105-122 BT)
#MSLP in any grid>=1020 hPa
mslcs=datamsl.sel(lat= slice(18,22), lon=slice(105,122), time=slice('1979-12-01T00:00:00.000000000','2020-02-29T00:00:00.000000000'))
mslcs=mslcs.sel(time=mslcs.time.dt.season=="DJF")['msl'].values
mslcs=np.mean(np.mean(mslcs, axis=2), axis=1)
thmsl=(np.mean(mslcs)+0.25*np.std(mslcs))
print('threshold CS mslp idx= ',thmsl, 'hPa')
mslcsidx=mslcs>=thmsl

threshold CS wind idx=  11.791974186897278 m/s
threshold CS mslp idx=  1018.0279010534286 hPa


In [11]:
#Calculate information in each CENS event
#duration  : number of consecutive days when the index pass the threshold
#idxmin : CENS index when CENS is strongest 
#time_start : first day when the index pass the threshold
#time_cens : day when CENS is strongest (vcens minimum)
#time_end : last day when the index pass the threshold
#csevt : CS no-CS classification

duration = []
idxmin = []
time_start = []
time_cens = []
time_end = []
csevt = []
running_count = 1
for i in range(len(censevt)-1):
    if censevt[i] == censevt[i+1]:
        running_count += 1
    else:
        if censevt[i]==1:
            duration.append(running_count)
            first=i-running_count+1
            time_start.append(time[first])
            idxmin.append(np.min(vcens[first:i+1]))
            arraymin=np.array(np.where(vcens[first:i+1]==np.min(vcens[first:i+1])))[0][0]+first
            time_cens.append(time[arraymin])
            time_end.append(time[i])
            if (wdcsidx[arraymin] or wdcsidx[arraymin-1]) and (mslcsidx[arraymin] or mslcsidx[arraymin-1]):
                csevt.append(1)
            else:
                csevt.append(0)
        running_count = 1

In [12]:
tglcenstab=pd.DataFrame({'time_cens': time_cens, 
                         'maxmagnitude': idxmin,
                         'duration': duration,
                         'time_start' : time_start,
                         'time_end': time_end,
                         'cs event' : csevt})

In [14]:
#Save to excel file
writer = pd.ExcelWriter('calculation_output\CENS Event 3 days Running Mean.xlsx')
tglcenstab.to_excel(writer, 'CENS_event')
writer.close()

In [15]:
#Statistic of CENS Event #3 days running mean
print('Threshold CENS index                                  : ', th, 'm/s')
print('Number of CENS Event (D(0)JF(1) 1979 - D(0)JF(1) 2019): ', np.shape(time_cens)[0], ' events')
print('Number of CENS-CS Event                               : ', np.sum(csevt), ' events')
print('Number of CENS-no CS Event                            : ', np.shape(time_cens)[0]-np.sum(csevt), ' events')
print('CENS Frequency per Season                             : ', np.shape(time_cens)[0]/(2020-1979), ' events')
print('CENS Frequency per Season CENS-CS Event               : ', np.sum(csevt)/(2020-1979), ' events')
print('CENS Frequency per Season CENS-no CS Event            : ', (np.shape(time_cens)[0]-np.sum(csevt))/(2020-1979), ' events')
print('CENS Index Average                                    : ', np.mean(idxmin), ' m/s')
print('CENS Index Average CENS-CS Event                      : ', np.mean(np.array(idxmin)[np.where(np.array(csevt)==1)]), ' m/s')
print('CENS Index Average CENS-no CS Event                   : ', np.mean(np.array(idxmin)[np.where(np.array(csevt)==0)]), ' m/s')
print('Duration Total                                        : ', np.sum(duration), ' days')
print('Duration Total CENS-CS Event                          : ', np.sum(np.array(duration)[np.where(np.array(csevt)==1)]), ' days')
print('Duration Total CENS-no CS Event                       : ', np.sum(np.array(duration)[np.where(np.array(csevt)==0)]), ' days')
print('Duration Average                                      : ', np.mean(duration), ' days')
print('Duration Average CENS-CS Event                        : ', np.mean(np.array(duration)[np.where(np.array(csevt)==1)]), ' days')
print('Duration Average CENS-no CS Event                     : ', np.mean(np.array(duration)[np.where(np.array(csevt)==0)]), ' days')

Threshold CENS index                                  :  -4.158942103385925 m/s
Number of CENS Event (D(0)JF(1) 1979 - D(0)JF(1) 2019):  117  events
Number of CENS-CS Event                               :  69  events
Number of CENS-no CS Event                            :  48  events
CENS Frequency per Season                             :  2.8536585365853657  events
CENS Frequency per Season CENS-CS Event               :  1.6829268292682926  events
CENS Frequency per Season CENS-no CS Event            :  1.170731707317073  events
CENS Index Average                                    :  -5.1063433  m/s
CENS Index Average CENS-CS Event                      :  -5.2526155  m/s
CENS Index Average CENS-no CS Event                   :  -4.8960757  m/s
Duration Total                                        :  596  days
Duration Total CENS-CS Event                          :  367  days
Duration Total CENS-no CS Event                       :  229  days
Duration Average                            